# Neresbot 0.1

Modelos:
* Convolução
* Long Short-Term Memory
* SARIMA

In [1]:
import pandas as pd
import requests
import json
from pprint import pprint
import yahooquery as yq
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

from tqdm import tqdm

In [2]:
# Função de padronização

def padronização(lista):
    máximo = max(lista)
    mínimo = min(lista)
    return list(map(lambda x: (x-mínimo)/(máximo-mínimo) if (máximo-mínimo) != 0 else 0.5,lista))

# Parâmetros Principais

In [3]:
# Baixar base de dados novamente (True) ou usar a preexistente (False)
download = True
# Usar menos dados e menos processamento
Parâmetros_reduzidos = True
# Criar o índice dos dataframes com base na coluna de datas (não recomendável)
indexing = False

Verif_Date = False

In [4]:
# Parâmetros Principais ##

# Tamanho das séries
período = 9
período = {0:'1d', 1:'5d', 2:'7d', 3:'60d', 4:'1mo', 5:'3mo', 6:'6mo', 7:'1y', 8:'2y', 9:'5y', 10:'10y', 11:'ytd', 12:'max'}[período]
print('período:',período)

# Tamanho de cada período
intervalo = 8
intervalo = {0:'1m', 1:'2m', 2:'5m', 3:'15m', 4:'30m', 5:'60m', 6:'90m', 7:'1h', 8:'1d', 9:'5d', 10:'1wk', 11:'1mo', 12:'3mo'}[intervalo]
print('intervalo:',intervalo)

# Variação identificada nos períodos seguintes
Variação = 0.1
# Períodos em que o aumento pode ocorrer
Prospecção = 10
# Rótulos negativos, para redução de preço na mesma proporção de aumento
Bidirecional = True
# Rótulos categóricos tentam prever em quantos dias a variação ocorre, do contrário, o resultado será 0 (queda), 1 (manuntenção) ou 2 (aumento).
Categórico = True

# Número deperíodos que compõem um array.
defasagem = 20

# Separação entre teste e validação.
# Não mantenha a soma de ambos maior que 0.5
Teste_porcento = 0.3
Valid_porcento = 0.2

# Número máximo de épocas de treinamento
épocas = 25
# Número de épocas que, consecutivamente iguais em termos de acurácia, encerram o treino
paciência = 70
# Se a acurácia desejada é alcançada, o treino é encerrado
Acurácia_desejada = 0.9

período: 5y
intervalo: 1d


In [5]:
# Metaparâmetros reduzidos


if Parâmetros_reduzidos:

    # Tamanho de cada período
    intervalo = 8
    intervalo = {0:'1m', 1:'2m', 2:'5m', 3:'15m', 4:'30m', 5:'60m', 6:'90m', 7:'1h', 8:'1d', 9:'5d', 10:'1wk', 11:'1mo', 12:'3mo'}[intervalo]
    print('intervalo:',intervalo)
    
    # Períodos em que o aumento pode ocorrer
    Prospecção = 5

    # Número deperíodos que compõem um array.
    defasagem = 7

    # Número máximo de épocas de treinamento
    épocas = 10
    # Número de épocas que, consecutivamente iguais em termos de acurácia, encerram o treino
    paciência = 5
    # Se a acurácia desejada é alcançada, o treino é encerrado
    Acurácia_desejada = 0.8

intervalo: 1d


# Download dos Dados

Os símbolos dos tickers são obtidos manualmente do site: https://www.dadosdemercado.com.br/bolsa/acoes

In [6]:
tickers = pd.read_csv("acoes-listadas.csv")

In [7]:
tickers

,Código,Nome
0,PETR4,Petrobras
1,MGLU3,Magazine Luiza
2,HAPV3,Hapvida
3,CIEL3,Cielo
4,B3SA3,B3
...,...,...
522,TKNO3,Tekno
523,COCE6,Coelce
524,MGEL3,Mangels
525,CTSA8,Santanense


https://yahooquery.dpguthrie.com/

É criado um dicionário que conterá, como chave, o símbolo do ticker e, como valor, o dataframe relacionando medidas e períodos (dias).

Em cada dataframe é adicionado o devido dia da semana, numericamente.

In [8]:
# Os símbolos das empresas brasileiras recebem o sufixo ".SA".

símbolos = tickers['Código'] + '.SA'

In [9]:
# Verificando símbolos inválidos.

print(yq.Ticker(símbolos,
                backoff_factor=1, 
                progress=True, 
                status_forcelist=[404, 429, 500, 502, 503, 504], 
                asynchronous=True
                ).invalid_symbols)

None


Ajuste dos dados:
* Download e armazenamento:
  * Os índices são resetados
  * É verificado se os dados possuem a coluna 'date':
    * É criada a coluna equivalente aos dias semanais
    * Armazenamento no dict dados1
    * Os dados são salvos na pasta 'dados'

In [10]:
# Download dos dados e alocação dos dataframes no dicionário "dados1"

if download:
    print(download)
    dados1 = {}
    falhas = []

    for ticker in tqdm(símbolos):
        yq.Ticker(ticker, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period=período, interval=intervalo,adj_timezone=True)
        except:
            falhas.append(ticker)
            #print(ticker,"falhou.")
        else:
            histórico = histórico.reset_index()
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dados1
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['weekday'] = padronização(histórico['weekday'])
                histórico['yearday'] = histórico['date'].dt.dayofyear # Cria dias da semana
                histórico['yearday'] = padronização(histórico['yearday'])
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date') # Organiza por data
                
                dados1[ticker] = histórico # Salva tudo em dados1
            else:
                #print(ticker,"não contém datas, mas contém:")
                #print(histórico.columns)
                None

    print(len(dados1),"tickers salvos com sucesso.")
    print(len(falhas),"tickers apresentaram falha.")

    Medidas = []
    for tckr in dados1:
        for col in dados1[tckr].columns:
            if col not in Medidas:
                Medidas.append(col)
                print(col)
    for tckr in dados1:
        for col in Medidas:
            if col not in dados1[tckr].columns:
                dados1[tckr][col] = 0

    # Verifica se a pasta de destino existe, se não, cria-a
    if not os.path.exists('dados'):
        os.makedirs('dados')
    # Itera pelo dicionário e salva cada dataframe como um arquivo CSV
    for tckr, df in dados1.items():
        caminho_arquivo = os.path.join('dados', f'{tckr}.csv')  # Caminho completo para o arquivo CSV
        df.to_csv(caminho_arquivo, index=False)  # Salva o dataframe como arquivo CSV

True


100%|██████████| 527/527 [00:00<00:00, 9937.77it/s]

0 tickers salvos com sucesso.
527 tickers apresentaram falha.


In [11]:
# Download reduzido, salvos em "dadosRe"

if download:
    dadosRe = {}
    falhas = []

    for ticker in tqdm(símbolos):
        query = yq.Ticker(ticker, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period='6m', interval=intervalo,adj_timezone=True)
        except:
            falhas.append(ticker)
            #print(ticker,"falhou.")
        else:
            histórico = histórico.reset_index()
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dadosRe
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['weekday'] = padronização(histórico['weekday'])
                histórico['yearday'] = histórico['date'].dt.dayofyear # Cria dias da semana
                histórico['yearday'] = padronização(histórico['yearday'])
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date') # Organiza por data
                
                dadosRe[ticker] = histórico # Salva tudo em dadosRe
            else:
                #print(ticker,"não contém datas, mas contém:")
                #print(histórico.columns)
                None

    print(len(dadosRe),"tickers salvos com sucesso.")
    print(len(falhas),"tickers apresentaram falha")

    Medidas = []
    for tckr in dadosRe:
        for col in dadosRe[tckr].columns:
            if col not in Medidas:
                Medidas.append(col)
                print(col)
    for tckr in dadosRe:
        for col in Medidas:
            if col not in dadosRe[tckr].columns:
                dadosRe[tckr][col] = 0

    # Verifica se a pasta de destino existe, se não, cria-a
    if not os.path.exists('dados'):
        os.makedirs('dados')
    # Itera pelo dicionário e salva cada dataframe como um arquivo CSV
    for tckr, df in dadosRe.items():
        caminho_arquivo = os.path.join('dados_reduzidos', f'{tckr}.csv')  # Caminho completo para o arquivo CSV
        df.to_csv(caminho_arquivo, index=False)  # Salva o dataframe como arquivo CSV

100%|██████████| 527/527 [04:08<00:00,  2.12it/s]


364 tickers salvos com sucesso.
58 tickers apresentaram falha
symbol
date
open
high
low
close
volume
adjclose
weekday
yearday
dividends


OSError: Cannot save file into a non-existent directory: 'dados_reduzidos'

In [ ]:

if download and Parâmetros_reduzidos:
    dados1 = {}
    for tkcr, hist in dadosRe.items():
        dados1[tckr] = hist

In [ ]:
# Carregando dados .csv

if not download:
    dados1 = {}  # Dicionário vazio para armazenar as tabelas

    if Parâmetros_reduzidos:
        pasta = 'dados_reduzidos'
    else:
        pasta = 'dados'

    # Percorre todos os arquivos na pasta
    for nome_arquivo in tqdm(os.listdir(pasta)):
        if nome_arquivo.endswith('.csv'):  # Verifica se o arquivo é um arquivo CSV
            caminho_arquivo = os.path.join('dados', nome_arquivo)  # Caminho completo para o arquivo

            histórico = pd.read_csv(caminho_arquivo)
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dadosRe
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['weekday'] = padronização(histórico['weekday'])
                histórico['yearday'] = histórico['date'].dt.dayofyear # Cria dias da semana
                histórico['yearday'] = padronização(histórico['yearday'])
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date') # Organiza por data
            
            dados1[os.path.splitext(nome_arquivo)[0]] = histórico

    # Imprime o dicionário com as tabelas
    print(len(dados1))

    print(dados1['PETR4.SA'])

    for i in dados1:
        print(i)

100%|██████████| 463/463 [00:04<00:00, 101.77it/s]

463
        symbol        date       open       high        low      close  \
0     PETR4.SA  2018-06-25  15.200000  15.790000  15.060000  15.700000   
1     PETR4.SA  2018-06-26  15.900000  16.120001  15.530000  16.040001   
2     PETR4.SA  2018-06-27  16.100000  16.980000  16.059999  16.549999   
3     PETR4.SA  2018-06-28  16.719999  17.070000  16.330000  16.600000   
4     PETR4.SA  2018-06-29  16.850000  17.190001  16.730000  17.190001   
...        ...         ...        ...        ...        ...        ...   
1234  PETR4.SA  2023-06-19  29.629999  30.459999  29.549999  30.420000   
1235  PETR4.SA  2023-06-20  30.299999  30.750000  29.900000  30.570000   
1236  PETR4.SA  2023-06-21  30.930000  32.000000  30.850000  31.850000   
1237  PETR4.SA  2023-06-22  31.459999  31.690001  31.080000  31.450001   
1238  PETR4.SA  2023-06-23  31.200001  31.280001  29.889999  30.160000   

        volume   adjclose  dividends  weekday   yearday  splits  
0     60347200   5.790563        0.0     

In [ ]:
# Verificando se todas as colunas 'date' são iguais 

ticker_divergentes = pd.DataFrame(columns=dados1.keys(), index=dados1.keys())

for column in dados1.keys():
    for index in dados1.keys():
        if dados1[column]['date'].equals(dados1[index]['date']):
            ticker_divergentes.at[index,column] = True
        else:
            ticker_divergentes.at[index,column] = False

ticker_divergentes.to_csv('ticker_divergentes.csv')
ticker_divergentes

,AALR3.SA,ABCB4.SA,ABEV3.SA,AERI3.SA,AESB3.SA,AFLT3.SA,AGRO3.SA,AGXY3.SA,AHEB3.SA,AHEB5.SA,...,VVEO3.SA,WEGE3.SA,WEST3.SA,WHRL3.SA,WHRL4.SA,WIZC3.SA,WLMM3.SA,WLMM4.SA,YDUQ3.SA,ZAMP3.SA
AALR3.SA,True,True,True,False,False,False,True,False,False,False,...,False,True,False,True,True,True,False,False,True,True
ABCB4.SA,True,True,True,False,False,False,True,False,False,False,...,False,True,False,True,True,True,False,False,True,True
ABEV3.SA,True,True,True,False,False,False,True,False,False,False,...,False,True,False,True,True,True,False,False,True,True
AERI3.SA,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AESB3.SA,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIZC3.SA,True,True,True,False,False,False,True,False,False,False,...,False,True,False,True,True,True,False,False,True,True
WLMM3.SA,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,True,False,False,False
WLMM4.SA,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
YDUQ3.SA,True,True,True,False,False,False,True,False,False,False,...,False,True,False,True,True,True,False,False,True,True


In [ ]:
sets_by_date = []

for tckr in dados1:
    hist = dados1[tckr]
    # A relação chave:valor é de tckr:hist

    if len(sets_by_date) == 0:
        sets_by_date.append({})
    
    set_index = 0
    Verif = True

    while Verif:
        if len(sets_by_date[set_index]) == 0:
            sets_by_date[set_index][tckr] = hist
            sets_by_date[set_index]['hist'] = hist['date']
            Verif = False
        else:
            if sets_by_date[set_index]['hist'].equals(hist['date']):
                sets_by_date[set_index][tckr] = hist
                Verif = False
            else:
                set_index += 1
                if set_index == len(sets_by_date):
                    sets_by_date.append({})


In [ ]:
# Verificação dos formatos das tabelas.

for sets in tqdm(sets_by_date):
    Len = {}
    for tckr in sets:
        if tckr != 'hist':
            Shape = str(sets[tckr].shape)
            if Shape not in Len.keys():
                Len[Shape] = 0
            Len[Shape] += 1

    print((len(sets)-1),'->',Len,)

100%|██████████| 80/80 [00:00<00:00, 26685.57it/s]

249 -> {'(1239, 12)': 249}
1 -> {'(648, 12)': 1}
1 -> {'(559, 12)': 1}
34 -> {'(1238, 12)': 34}
1 -> {'(477, 12)': 1}
77 -> {'(1237, 12)': 77}
1 -> {'(550, 12)': 1}
1 -> {'(769, 12)': 1}
1 -> {'(733, 12)': 1}
1 -> {'(1236, 12)': 1}
3 -> {'(475, 12)': 3}
1 -> {'(579, 12)': 1}
1 -> {'(616, 12)': 1}
1 -> {'(1196, 12)': 1}
1 -> {'(545, 12)': 1}
1 -> {'(884, 12)': 1}
8 -> {'(1238, 12)': 8}
2 -> {'(589, 12)': 2}
1 -> {'(673, 12)': 1}
2 -> {'(473, 12)': 2}
1 -> {'(868, 12)': 1}
2 -> {'(651, 12)': 2}
1 -> {'(484, 12)': 1}
1 -> {'(900, 12)': 1}
1 -> {'(584, 12)': 1}
6 -> {'(1240, 12)': 6}
1 -> {'(684, 12)': 1}
2 -> {'(537, 12)': 2}
2 -> {'(480, 12)': 2}
2 -> {'(680, 12)': 2}
1 -> {'(460, 12)': 1}
2 -> {'(713, 12)': 2}
1 -> {'(516, 12)': 1}
2 -> {'(587, 12)': 2}
1 -> {'(1045, 12)': 1}
1 -> {'(594, 12)': 1}
1 -> {'(539, 12)': 1}
1 -> {'(669, 12)': 1}
1 -> {'(601, 12)': 1}
1 -> {'(532, 12)': 1}
1 -> {'(67, 12)': 1}
2 -> {'(592, 12)': 2}
1 -> {'(691, 12)': 1}
1 -> {'(463, 12)': 1}
1 -> {'(467, 12)'

# Convolução

## Rótulos

Cada linha analisa se há aumento expressivo nos próximos 5 períodos.

Se não houver, o rótulo será 0; Caso contrário, será o número de dias em que o aumento ocorre.  
Há uma variável ajustável para determinar o nível do aumento.

In [17]:
dados2 = sets_by_date[0]
if 'hist' in dados2.keys():
    del dados2['hist']

In [18]:
# Verificando se todas as colunas 'date' são iguais 

if Verif_Date:
    ticker_divergentes = pd.DataFrame(columns=dados2.keys(), index=dados2.keys())

    for column in dados2.keys():
        for index in dados2.keys():
            if dados1[column]['date'].equals(dados2[index]['date']):
                ticker_divergentes.at[index,column] = True
            else:
                ticker_divergentes.at[index,column] = False

    ticker_divergentes.to_csv('ticker_divergentes.csv')
    ticker_divergentes

In [19]:
print('Parâmetros_reduzidos:',Parâmetros_reduzidos)
print('Prospecção:',Prospecção)
print('Bidirecional:',Bidirecional)
print('Categórico:',Categórico)

Parâmetros_reduzidos: True
Prospecção: 5
Bidirecional: True
Categórico: True


In [20]:
# 2 - Valores dos Rótulos, separando as buscaspositivas e negativas
Verbose = True
print(f'') if Verbose else None

print('SE BIDIRECIONAL: OS VALORES MAIS ALTOS SERÃO O DOBRO DA PROSPECÇÃO.')
# Coluna para os rótulos

for i,j in dados2.items():
    j['rótulos'] = 0
    j['compar'] = 0

# Valores para os rótulos

for tckr, df in tqdm(dados2.items()):                  # Para cada ticker, tabela:                      # reseta o índice da tabela
    print() if Verbose else None
    print('Ticker:',tckr) if Verbose else None
    
    for i, row in df.iterrows():
        price_now = row['close']
        price_contester_high = row['close']
        price_contester_high_id = i
        price_contester_low = row['close']
        price_contester_low_id = i
        print(' i:',i,'; price_now e contests:',price_now) if Verbose else None


        # Procurando positivos.
        print(' Procurando aumentos:') if Verbose else None
        for j in range(i+1, i+1+Prospecção):
            
            # Se J ainda é dentro da tabela
            if j < len(df):
                print(f'  j:{j}') if Verbose else None
                j_high = df.loc[j, 'high']
                print(f'   high de j={j}: {j_high}.') if Verbose else None
                
                # se o preço avaliado é maior ou igual ao aumento esperado:
                if (j_high >= (1 + Variação)*price_now) and (j_high > price_contester_high):
                    print(f'     j_high ({j_high}) supera a variação esperada ({(1 + Variação)*price_now}) e o último maior ({price_contester_high}).') if Verbose else None
                    if Categórico:
                        dados2[tckr].loc[i, 'rótulos'] = j - i
                    else:
                        dados2[tckr].loc[i, 'rótulos'] = 1
                    dados2[tckr].loc[i, 'compar'] = j_high
                    price_contester_high = j_high
                    price_contester_high_id = j
                    temp = dados2[tckr].loc[i, 'rótulos']
                    print(f'     Rótulo atualizado: {temp}')
                    print(f'     O último maior preço para o id {i} agora é {price_contester_high}, do id {j}.') if Verbose else None
                else:
                    temp = dados2[tckr].loc[i, 'rótulos']
                    print(f'     Rótulo permanece: {temp}') if Verbose else None
            else:
                print('   j ultrapassa o limite da tabela')


        
        # Iteração entre os próximos "Prospecção" itens, procurando negativos.
        if Bidirecional:
            print(' Procurando aumentos:') if Verbose else None
            for j in range(i+1, i+1+Prospecção):
                
                # Se J ainda é dentro da tabela
                if j < len(df):
                    print(f'  j:{j}') if Verbose else None
                    j_low = df.loc[j, 'low']
                    print(f'   low de j={j}: {j_low}.') if Verbose else None

                    # se o preço avaliado é menor ou igual à redução esperada e é bidirecional:
                    if (j_low <= (1-Variação)*price_now) and (j_low < price_contester_low) and Bidirecional:
                        print(f'     post_low ({j_low}) supera a variação esperada ({(1 + Variação)*price_now}) e o último maior ({price_contester_low}).') if Verbose else None
                        if Categórico:
                            dados2[tckr].loc[i, 'rótulos'] = i - j
                        else:
                            dados2[tckr].loc[i, 'rótulos'] = -1
                        dados2[tckr].loc[i, 'compar'] = j_low
                        price_contester_low = j_low
                        price_contester_low_id = j
                        temp = dados2[tckr].loc[i, 'rótulos']
                        print(f'     Rótulo atualizado: {temp}')
                        print(f'     O último menor preço para o id {i} agora é {price_contester_low}, do id {j}.') if Verbose else None
                    else:
                        temp = dados2[tckr].loc[i, 'rótulos']
                        print(f'     Rótulo permanece: {temp}')
                else:
                    print('   j ultrapassa o limite da tabela')
                
                

        if dados2[tckr].loc[i, 'rótulos'] == 0:
            print(f'  Nenhum id entre {i+1} e {j} apresentou valores consideráveis')
            print(f'    j_high depois: {j_high}') if Verbose else None
            print(f'    j_low depois: {j_low}') if Verbose else None


        # Remoção de rótulos negativos
        if Bidirecional:
            if Categórico:
                dados2[tckr].loc[i, 'rótulos'] += Prospecção
            else:
                dados2[tckr].loc[i, 'rótulos'] += 1
        temp = dados2[tckr].loc[i, 'rótulos']
        print(f'    Para o id ({i}) o rótulo definitivo é {temp}') if Verbose else None

        if Verbose:
            if (((dados2[tckr].loc[i, 'rótulos'] != Prospecção) and (Categórico)) or (dados2[tckr].loc[i, 'rótulos'] != 1)) and (tckr == 'ITSA4.SA'):
                print(row['symbol'], dados2[tckr].loc[i, 'rótulos'], (((j_high/price_now)-1)*100), (((j_high/price_now)-1)*100))
                print(row['symbol'], dados2[tckr].loc[i, 'rótulos'], (((j_low/price_now)-1)*100), (((j_low/price_now)-1)*100))
    
    # print(df.head()) #??


SE BIDIRECIONAL: OS VALORES MAIS ALTOS SERÃO O DOBRO DA PROSPECÇÃO.


  0%|          | 0/249 [00:00<?, ?it/s]


Ticker: AALR3.SA
 i: 0 ; price_now e contests: 12.949999809265137
  j: 1
   high de j=1: 13.09000015258789.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 2
   high de j=2: 12.90999984741211.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 3
   high de j=3: 12.979999542236328.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 4
   high de j=4: 13.140000343322754.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 5
   high de j=5: 13.050000190734863.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 1
   low de j=1: 12.600000381469728.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 2
   low de j=2: 12.720000267028809.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 3
   low de j=3: 12.760000228881836.
    j não atravessa o limite da tabela,
     Rótulo permanece: 0
  j: 4
   low de j=4: 12.81999969482422.
    j não atravessa o limite da t

  0%|          | 0/249 [00:01<?, ?it/s]


KeyError: 1239

In [ ]:
print('  Rótulos')
for tckr in dados2:
    print(tckr+':', dados2[tckr]['rótulos'].min(), dados2[tckr]['rótulos'].max())

In [ ]:
for tckr in dados2:
    print(dados2[tckr].tail(4))

In [ ]:
print('  Dividendos')
for tckr in dados2:
    print(tckr+':',dados2[tckr]['dividends'].max())

## Transformação das matrizes: Separando os dias como objetos numpy.

Classes que contenham as matrizes, os rótulos e a identificação do dia.

In [ ]:
# Todos os dataframes:

df_concatenado = pd.concat(dados2.values(), axis=1)

In [ ]:
df_concatenado.to_csv('df_concatenado.csv',index=True)

O último dia não deve estar incluso no treinamento por não possuir rótulo.

Dimensões:
* Medidas:
  * open
  * high
  * low
  * close
  * volume
  * adjclose
  * weekday
  * yearday
  * dividends
  * splits
* tickers
* Dias (atual e anteriores)

In [ ]:
# Adição de colunas "necessárias"
# Remoção das colunas "desnecessárias"

 Colunas_Necessárias = ['open',
                        'high',
                        'low', 
                        'close', 
                        'volume', 
                        'adjclose', 
                        'weekday', 
                        'yearday', 
                        'dividends', 
                        'splits', 
                        'rótulos']

dados3 = {}

for tckr in dados2.keys():
    for c in Colunas_Necessárias:
        if c no in dados2[tckr].columns:
            dados2[tckr][c] = 0

for tckr in dados2.keys():
    dados3[tckr] = dados2[tckr][Colunas_Necessárias]

In [ ]:
dados3['AALR3.SA'].columns

In [ ]:
# Tendo certeza de que todas as tabelas têm o mesmo tamanho.

for t in dados3.keys():
    for k in dados3.keys():
        Série = len(dados3[t])
        dif = len(dados3[t])-len(dados3[k])
        if dif != 0:
            print(t,'-',k,'=',dif)

In [ ]:
Série

Há uma variável ajustável para determinar quantos dias serão olhados para trás.

In [ ]:
# Criação lista completa

Tensor = []

Rótulos = {}
for tckr in dados3.keys():
    Rótulos[tckr] = []

for i in range(Série-Prospecção):
    lvl1 = []
    for tckr in dados3.keys():
        lvl2 = []
        for coluna in dados3[tckr].columns:
            if coluna != 'rótulos':
                lvl3 = []
                for j in range(i - defasagem, i):
                    if j >= 0:
                        k = j
                    else:
                        k = 0
                    lvl3.append(dados3[tckr].at[k,coluna])
                    if coluna not in ['weekday','yearday']:
                        lvl3p = padronização(lvl3)
            lvl2.append(lvl3p)
        lvl1.append(lvl2)
        Rótulos[tckr].append(dados3[tckr].at[i,'rótulos'])
    Tensor.append(lvl1)

## Separação entre Treino, Teste e Validação.

É preciso garantir que as matrizes sejam embaralhadas, mas que os rótulos permaneçam atrelados adequadamente.

In [ ]:
type(Tensor)

In [ ]:
Treino_tensor = []
Teste_tensor = []
Valid_tensor = []

Treino_rótulo = {}
for tckr in dados3.keys():
    Treino_rótulo[tckr] = []

Teste_rótulo = {}
for tckr in dados3.keys():
    Teste_rótulo[tckr] = []

Valid_rótulo = {}
for tckr in dados3.keys():
    Valid_rótulo[tckr] = []

id_teste_valid = random.sample(list(range(len(Tensor))), int(round(len(Tensor)*(Teste_porcento+Valid_porcento), 0)))

id_valid = random.sample(id_teste_valid, int(round(len(Tensor)*Valid_porcendo, 0)))

for i in range(len(Tensor)):
    if i in id_valid:
        Valid_tensor.append(Tensor[i])
        for tckr in Valid_rótulo.keys():
            Valid_rótulo[tckr].append(Rótulos[tckr][i])
            
    elif i in id_teste_valid:
        Teste_tensor.append(Tensor[i])
        for tckr in Teste_rótulo.keys():
            Teste_rótulo[tckr].append(Rótulos[tckr][i])

    else:
        Treino_tensor.append(Tensor[i])
        for tckr in Treino_rótulo.keys():
            Treino_rótulo[tckr].append(Rótulos[tckr][i])

Treino_tensor = np.array(Treino_tensor)
Teste_tensor = np.array(Teste_tensor)
Valid_tensor = np.array(Valid_tensor)

for tckr in Treino_rótulo.keys():
    Treino_rótulo[tckr] = np.array(Treino_rótulo[tckr])
for tckr in Teste_rótulo.keys():
    Teste_rótulo[tckr] = np.array(Teste_rótulo[tckr])
for tckr in Valid_rótulo.keys():
    Valid_rótulo[tckr] = np.array(Valid_rótulo[tckr])

In [ ]:
print(Treino_tensor.shape)
print(Teste_tensor.shape)
print(Valid_tensor.shape)

print()

print(len(Treino_rótulo),len(Treino_rótulo['PETR4.SA']))
print(len(Teste_rótulo),len(Teste_rótulo['PETR4.SA']))
print(len(Valid_rótulo),len(Valid_rótulo['PETR4.SA']))

## Modelo

* Para cada ticker (a partir do dicionário de rótulos):  
    * O modelo é treinado seguindo aquele ticker como rótulo (com uma função)  
    * O modelo é validado  
    * Se o modelo é válido:  
        * A previsão é tentada:  
        * Se o resultado é diferente de zero E a probabilidade é alta:  
                * O deploy é salvo

Exemplo com um ticker: PTR4.SA

In [ ]:
Shape = Treino_tensor.shape[1:]
Ativação = 'relu'

In [ ]:
# Cria o objeto de sequência de camadas
modelo = models.Sequential()

modelo.add(layers.Conv2D(64, (2, 2), activation = 'relu', input_shape = Shape))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Conv2D(64, (2, 2), activation = 'relu'))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Conv2D(64, (1, 1), activation = 'relu'))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Flatten())

modelo.add(layers.Dense(64, activation = 'relu'))

modelo.add(layers.Dense(32, activation = 'relu'))

modelo.add(layers.Dense((Prospecção+1), activation = 'softmax'))

modelo.summary()

In [ ]:
modelo.compile(optimizer = 'adam', 
                   loss = 'sparse_categorical_crossentropy', 
                   metrics = ['accuracy'])

In [ ]:
class AccuracyThresholdCallback(Callback):
    def __init__(self, accuracy_threshold):
        super(AccuracyThresholdCallback, self).__init__()
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= self.accuracy_threshold:
            self.model.stop_training = True
            print(f"\nAtingiu a acurácia desejada ({self.accuracy_threshold}). Treinamento interrompido.")

In [ ]:
# Criando um parâmetro de parada com o objeto EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', mode='max', patience=paciência, verbose=1)

# Salvando a melhor época.
checkpoint = ModelCheckpoint('Modelos\\melhor_CNN.h5', monitor='val_accuracy', mode='max', save_best_only=True)

# Criar o objeto do callback personalizado
accuracy_callback = AccuracyThresholdCallback(Acurácia_desejada)

In [ ]:
# Treinamento e teste

history = modelo.fit(Treino_tensor, 
                         Treino_rótulo['PETR4.SA'], 
                         epochs = 5, 
                         validation_data = (Teste_tensor, Teste_rótulo['PETR4.SA']),
                         callbacks=[early_stopping, checkpoint])

In [ ]:
modelo.evaluate(Valid_tensor, Valid_rótulo['PETR4.SA'], verbose = 2)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = modelo.evaluate(Valid_tensor, Valid_rótulo['PETR4.SA'], verbose=2)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

test_predictions = modelo.predict(Valid_tensor)

test_predicted_labels = np.argmax(test_predictions, axis=1)
print(test_predicted_labels)

test_true_labels = np.array(Valid_rótulo['PETR4.SA'])
print(test_true_labels)

cm = confusion_matrix(test_true_labels, test_predicted_labels)

cmd = ConfusionMatrixDisplay(confusion_matrix=cm)

cmd.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
plt.title('PETR4.SA')
plt.show()

In [ ]:
shape_saída = (Prospecção*2)+1 if Categórico and Bidirecional else \
              Prospecção+1 if Categórico else \
              3 if Bidirecional else \
              2


Modelos = {}

for tckr in tqdm(Rótulos.keys()):
    # Cria o objeto de sequência de camadas
    
    Modelos[tckr] = models.Sequential()

    Modelos[tckr].add(layers.Conv2D(64, (2, 2), activation = 'relu', input_shape = Shape))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Conv2D(64, (2, 2), activation = 'relu'))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Conv2D(64, (1, 1), activation = 'relu'))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Flatten())

    Modelos[tckr].add(layers.Dense(64, activation = 'relu'))

    Modelos[tckr].add(layers.Dense(32, activation = 'relu'))

    Modelos[tckr].add(layers.Dense(shape_saída, activation = 'softmax'))

    Modelos[tckr].compile(optimizer = 'adam', 
                    loss = 'sparse_categorical_crossentropy', 
                    metrics = ['accuracy'])

    # Treinamento e teste

    history = Modelos[tckr].fit(Treino_tensor, 
                            Treino_rótulo[tckr], 
                            epochs = épocas, 
                            validation_data = (Teste_tensor, Teste_rótulo[tckr]),
                            callbacks = [early_stopping, checkpoint], # accuracy_callback
                            verbose = 0)
    
    Modelos[tckr].evaluate(Valid_tensor, Valid_rótulo[tckr], verbose = 0)
    
    test_predictions = Modelos[tckr].predict(Valid_tensor)

    test_predicted_labels = np.argmax(test_predictions, axis=1)

    test_true_labels = np.array(Valid_rótulo[tckr])

    cm = confusion_matrix(test_true_labels, test_predicted_labels)

    cmd = ConfusionMatrixDisplay(confusion_matrix=cm)

    cmd.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')

    plt.title(tckr+' ('+str(len(test_true_labels))+')')

    plt.savefig('conv_confusion_matrix\\' + tckr+'.png')

* Versão reduzida carrega dados curtos
* Versão download baixa dados curtos e dados extensos
* Os dados são salvos e carregados, mesmo na versão download
* Versão carregamento carrega os dados de acordo com a versão
* Metaparâmetro de saída: categórica (uma para cada dia) ou binária (se há aumento, tá valendo)
* Metaparâmetro de saída: unidirecional ou bidirecional.
* Objeto de tracking para averiguar se os dados não foram desalinhados
* Salvar a acertividade dos modelos por ticker
* Verificar porque alguns gráficos de calor têm menos observações.